In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

KeyboardInterrupt: 

In [ ]:
data = pd.read_csv('../../Datasets/Retail/Retail_train_data.csv')

In [ ]:
data.shape

In [ ]:
data['Store'].value_counts()

In [ ]:
data['Sales'].loc[(data['Open']==0) & (data['Store']== 1)]

In [ ]:
data.drop(data[data['Open']==0].index,inplace=True)

In [ ]:
data.head()

In [ ]:
data['Open'].unique

In [ ]:
for i in ['Open','Promo','StateHoliday','SchoolHoliday']:
    print(data[i].nunique)

In [ ]:
data.info()

In [ ]:
data.describe(include='all')

In [ ]:
data.isnull().sum()

In [ ]:
from statsmodels.tsa.stattools import adfuller


In [ ]:
data_index = data.loc[data['Store']==1].set_index('Date')

In [ ]:
data_index.head()

In [ ]:
customers = data_index[['Customers']].loc[data_index['Store'] == 1]
sales = data_index[['Sales']].loc[data_index['Store'] == 1]

In [ ]:
sales

In [ ]:

plt.plot(sales)
plt.show()

In [ ]:
def test_stationary(timeseries):

    # determine rolling stats.
    movingAvg=timeseries.rolling(window=7).mean()
    movingStd=timeseries.rolling(window=7).std()

    # plot rolling stats.
    plt.plot(timeseries,color='blue',label='Original')
    plt.plot(movingAvg,color='red',label='Rolling Avg')
    plt.plot(movingStd,color='black',label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Time series Stats')
    plt.show()


    # Perform Dickey fuller test
    print('results: ')
    dickey_Fuller_Test = adfuller(timeseries)
    dftestOutput = pd.Series(dickey_Fuller_Test[0:4],index=['Test Stat','p-value','#Lags Used','Number of Observations used'])
    for key,value in dickey_Fuller_Test[4].items():
        dftestOutput['critical values(%s)' %key] = value
    print(dftestOutput)

In [ ]:
test_stationary(sales)

In [ ]:
sales_log = np.log(sales)

In [ ]:
sales_log

In [ ]:
test_stationary(sales_log)

In [ ]:
sales_log_movingAvg = sales_log.rolling(window=7).mean()
sales_log_movingStd = sales_log.rolling(window=7).std()

In [ ]:
sales_new = sales_log - sales_log_movingAvg

In [ ]:
sales_new.head(25)

In [ ]:
sales_new.dropna(inplace=True)
test_stationary(sales_new)

In [ ]:
sales_TimeShift = sales_log-sales_log.shift()

In [ ]:
sales_TimeShift

In [ ]:
sales_TimeShift.dropna(inplace=True)

In [ ]:
test_stationary(sales_TimeShift)

In [ ]:
data.sort_values(by='Date',ascending=True,inplace=True)

In [ ]:
data_store1 = data[['Date','Sales']].loc[data['Store']==1]

In [ ]:
data_store1['Sales'].dropna().plot(figsize=(20,5))

In [ ]:
data_store1['rolling_mean'] = data_store1['Sales'].rolling(window=3).mean()

In [ ]:
data_store1['Sales'].isna().sum()

In [ ]:
data_store1.dropna(inplace=True,axis=0)

In [ ]:
for i in data_store1.loc[data_store1['Sales']==0].index:
    data_store1['Sales'].loc[i]=data_store1['rolling_mean'].loc[i]

In [ ]:
data_store1['Sales'].plot(figsize=(20,5))

In [ ]:
def create_timeseries_data(timeseries,previous_lag):
    X = []
    y = []
    prev_data = []
    length = len(timeseries) - (previous_lag - 1)
    for i in range(length):
        for j in range(previous_lag):
            if(j==previous_lag-1):
                X.append(prev_data.copy())
                y.append(timeseries[i+j])
                prev_data.clear()
            else:
                prev_data.append(timeseries[i+j])
    return np.array(X),np.array(y)

In [ ]:
previous_lag = 4
X,y = create_timeseries_data(data_store1['Sales'].values,previous_lag)

In [ ]:
X.shape,y.shape

In [ ]:
X = X.reshape(X.shape[0],previous_lag-1,1)
#y = y.reshape(y.shape[0],1)

In [ ]:
X.shape,y.shape

In [ ]:
from tensorflow.keras import models, layers

In [ ]:
model = models.Sequential()
model.add(layers.LSTM(500,return_sequences = True, activation='relu',input_shape=X.shape[1:]))
model.add(layers.LSTM(500,activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer='ADAM',loss='mse')
model.summary()

In [ ]:
model_hist = model.fit(X,y,epochs=500,verbose=0)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(model_hist.history['loss'])

In [ ]:
x_input = X[905]

In [ ]:
x_input = x_input.reshape(3,)

In [ ]:
for i in range(100):
    print(i)
    x_ip = x_input[i:i+3]
    y_pred = model.predict(x_ip.reshape(1,x_ip.shape[0],1))
    x_input = np.append(x_input,y_pred[0][0])

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(data_store1['Sales'],color='blue')

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(x_input,color='blue')

In [ ]:
data_ANN = data.copy()

In [ ]:
data_ANN.describe(include='all')

In [ ]:
date = data_ANN['Date'].values

In [ ]:
date_month = []
for i in date:
    date_month.append(int(i.split('-')[1]))

In [ ]:
date_month = np.array(date_month)

In [ ]:
data_ANN['Month'] = date_month

In [ ]:
data_ANN.drop('Date',axis=1,inplace=True)

In [ ]:
data_ANN

In [ ]:
data_ANN.describe()

In [ ]:
data_ANN_Target = data_ANN['Sales'].copy()
data_ANN.drop(['Sales','StateHoliday'],inplace=True,axis=1)

In [ ]:
data_ANN.info()

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(data_ANN,data_ANN_Target,test_size=0.8,random_state=42)

In [ ]:
from tensorflow.keras import models, layers

In [ ]:
X_train.shape

In [ ]:
model_ANN = models.Sequential()
model_ANN.add(layers.Dense(500,activation = 'relu'))
model_ANN.add(layers.Dropout(0.2))

model_ANN.add(layers.Dense(500,activation = 'relu'))
model_ANN.add(layers.Dropout(0.3))

model_ANN.add(layers.Dense(500,activation = 'relu'))
model_ANN.add(layers.Dropout(0.4))

model_ANN.add(layers.Dense(1,activation= 'linear'))

model_ANN.compile(optimizer='ADAM',loss='mse')

In [ ]:
model_hist = model_ANN.fit(X_train,y_train,epochs=100,batch_size=256,validation_split=0.2)